In [ ]:
import requests
import time
import json
from datetime import datetime

API_KEY = "MhlwpLEgaTMO0kxeTAZnx1tLsUeOAGEC"
BASE = "https://app.ticketmaster.com/discovery/v2/events.json"

def fetch_all_events(keyword=None, geo=None, country_code="US"):
    """Fetches ALL Ticketmaster events with pagination."""
    
    events = []
    page = 0

    while True:
        params = {
            "apikey": API_KEY,
            "size": 200,
            "page": page,
            "countryCode": country_code
        }

        if keyword:
            params["keyword"] = keyword
        
        if geo:
            params["geoPoint"] = geo

        response = requests.get(BASE, params=params)
        data = response.json()

        # No events returned → break
        if "_embedded" not in data:
            break

        new_events = data["_embedded"]["events"]
        events.extend(new_events)

        print(f"Fetched page {page} → {len(new_events)} events")
        
        total_pages = data["page"]["totalPages"]

        if page >= total_pages - 1:
            print("Completed pulling all pages!")
            break

        page += 1
        time.sleep(0.1)  # small delay to stay safe

    # Save to disk
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    with open(f"ticketmaster_events_{timestamp}.json", "w") as f:
        json.dump(events, f, indent=2)

    print(f"Saved {len(events)} events.")
    return events


if __name__ == "__main__":
    fetch_all_events()


Fetched page 0 → 200 events
Fetched page 1 → 200 events
Fetched page 2 → 200 events
Fetched page 3 → 200 events
Fetched page 4 → 200 events
Saved 1000 events.


In [1]:
import requests
import time
import json
from datetime import datetime
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn
from rich.table import Table

from pathlib import Path

# Ensure directory exists
Path("data/raw").mkdir(parents=True, exist_ok=True)


# ---------------------------------------
# Rich Console
# ---------------------------------------
console = Console()

# ---------------------------------------
# Ticketmaster API Config
# ---------------------------------------
API_KEY = "MhlwpLEgaTMO0kxeTAZnx1tLsUeOAGEC"
BASE = "https://app.ticketmaster.com/discovery/v2/events.json"

SEGMENT_MUSIC = "KZFzniwnSyZfZ7v7nJ"  # Ticketmaster music segment ID
DAILY_LIMIT = 5000
SAFE_RPS = 1.67
SLEEP_TIME = 1 / SAFE_RPS  # ~0.6 seconds

# ---------------------------------------
# USA Geohash Coverage Grid (36 cells)
# ---------------------------------------
US_GEOHASHES = [
    "dr5","dr7","dr9","drc","drf","drg",
    "dj8", "djd","djf","djg","djk",
    "dp3","dp5","dp8","dp9","dpe","dph",
    "dq2","dq5","dq8","dq9",
    "9q5","9q8","9qb","9qc","9qh","9qk",
    "9xj","9xk","9xm","9xs",
    "9yf","9yh","9yj"
]

# ---------------------------------------
# Global Request Counter
# ---------------------------------------
request_count = 200


# =======================================
# METADATA EXTRACTION
# =======================================
def extract_metadata(raw_event):
    """
    Extract structured metadata from Ticketmaster event object.
    Includes:
    - Event identifiers/date/status/onsale
    - Artist info
    - Venue info including lat/lon/postal
    - Classifications (segment/genre/subgenre)
    - Price ranges
    - Promoter
    - Seatmap
    
    NOTE: Uses get() and default values (like [{}]) to handle missing keys gracefully.
    """

    # --- Event Core ---
    event_id = raw_event.get("id")
    event_name = raw_event.get("name")
    event_url = raw_event.get("url")

    # --- Date Fields ---
    event_date = raw_event.get("dates", {}).get("start", {}).get("dateTime")
    timezone = raw_event.get("dates", {}).get("timezone")
    event_status = raw_event.get("dates", {}).get("status", {}).get("code")

    onsale_date = raw_event.get("sales", {}).get("public", {}).get("startDateTime")

    # --- Embedded Check (Crucial for Artist/Venue Data) ---
    embedded = raw_event.get("_embedded", {})

    # --- Artist ---
    attractions = embedded.get("attractions", [{}])
    artist = attractions[0] if attractions else {} # Safe access to first artist

    artist_id = artist.get("id")
    artist_name = artist.get("name")
    artist_url = artist.get("url")

    # --- Venue ---
    venue_list = embedded.get("venues", [{}])
    venue = venue_list[0] if venue_list else {} # Safe access to first venue

    venue_id = venue.get("id")
    venue_name = venue.get("name")

    city = venue.get("city", {}).get("name")
    state_code = venue.get("state", {}).get("stateCode")
    postal_code = venue.get("postalCode")
    country_code = venue.get("country", {}).get("countryCode")

    location = venue.get("location", {})
    latitude = location.get("latitude")
    longitude = location.get("longitude")

    capacity = venue.get("capacity") # Not always present

    # --- Pricing ---
    price_ranges = raw_event.get("priceRanges", [{}])
    price_min = price_ranges[0].get("min") if price_ranges else None
    price_max = price_ranges[0].get("max") if price_ranges else None
    currency = price_ranges[0].get("currency") if price_ranges else None

    # --- Classifications ---
    class_info = raw_event.get("classifications", [{}])[0]

    segment = class_info.get("segment", {}).get("name")
    genre = class_info.get("genre", {}).get("name")
    subgenre = class_info.get("subGenre", {}).get("name")
    type_name = class_info.get("type", {}).get("name")
    subtype = class_info.get("subType", {}).get("name")

    # --- Promoter ---
    promoter = raw_event.get("promoter", {})
    promoter_name = promoter.get("name")
    promoter_id = promoter.get("id")

    # --- Seatmap ---
    seatmap_url = raw_event.get("seatmap", {}).get("staticUrl")

    return {
        "event_id": event_id,
        "event_name": event_name,
        "event_url": event_url,

        "event_date": event_date,
        "timezone": timezone,
        "event_status": event_status,
        "onsale_date": onsale_date,

        "artist_id": artist_id,
        "artist_name": artist_name,
        "artist_url": artist_url,

        "venue_id": venue_id,
        "venue_name": venue_name,
        "city": city,
        "state_code": state_code,
        "postal_code": postal_code,
        "country_code": country_code,
        "latitude": latitude,
        "longitude": longitude,
        "capacity": capacity,

        "price_min": price_min,
        "price_max": price_max,
        "currency": currency,

        "segment": segment,
        "genre": genre,
        "subgenre": subgenre,
        "type": type_name,
        "subtype": subtype,

        "promoter_name": promoter_name,
        "promoter_id": promoter_id,

        "seatmap_url": seatmap_url
    }


# =======================================
# REQUEST WRAPPER WITH COUNTER
# =======================================
def make_request(url, params):
    """Handles request counting, warnings, and actual GET call."""
    global request_count
    request_count += 1

    if request_count % 50 == 0:
        console.print(f"[yellow]Request count: {request_count}/{DAILY_LIMIT}[/yellow]")

    if request_count >= DAILY_LIMIT * 0.9:
        console.print(f"[bold red]⚠ NEAR DAILY LIMIT: {request_count} requests[/bold red]")

    return requests.get(url, params=params)


# =======================================
# FETCH EVENTS PER GEOHASH
# =======================================
def fetch_events_for_geohash(geohash):
    cleaned_events = []
    total_raw = 0
    page = 0

    while True:
        params = {
            "apikey": API_KEY,
            "segmentId": SEGMENT_MUSIC,
            "geoPoint": geohash,
            "radius": 200,
            "size": 200,
            "page": page
        }

        response = make_request(BASE, params)

        if response.status_code == 429:
            console.print("[bold red]Rate limit hit: 429 → Sleeping 5 seconds[/bold red]")
            time.sleep(5)
            continue

        if response.status_code != 200:
            console.print(f"[red]Error {response.status_code} on geohash {geohash} page {page}: {response.text}[/red]")
            break

        data = response.json()

        # Simple check for the core event list existence
        if "_embedded" not in data or "events" not in data["_embedded"]:
            break

        raw_events = data["_embedded"]["events"]
        total_raw += len(raw_events)

        for raw_event in raw_events:
            cleaned_events.append(extract_metadata(raw_event))

        total_pages = data["page"]["totalPages"]
        if page >= total_pages - 1:
            break

        page += 1
        time.sleep(SLEEP_TIME)

    return cleaned_events, total_raw


# =======================================
# MAIN SCRAPER FUNCTION
# =======================================
def fetch_all_us_music_events():
    console.print("\n[bold cyan]🎵 Fetching ALL USA Music Events (Geohash Grid)[/bold cyan]\n")

    all_events = {}
    total_raw = 0

    with Progress(
        SpinnerColumn(),
        TextColumn("[progress.description]{task.description}"),
        BarColumn(),
        TextColumn("{task.completed}/{task.total} geohashes done"),
        transient=True,
    ) as progress:

        task = progress.add_task("Processing geohashes...", total=len(US_GEOHASHES))

        for gh in US_GEOHASHES:

            events, raw_count = fetch_events_for_geohash(gh)
            total_raw += raw_count

            if raw_count == 0:
                console.print(f"[red]⚠ No events returned for geohash {gh}![/red]")

            for ev in events:
                all_events[ev["event_id"]] = ev  # dedupe

            console.print(f"[green]Geohash {gh}: {len(events)} unique events[/green]")

            progress.update(task, advance=1)

    final_events = list(all_events.values())
    
    Path("data/raw").mkdir(parents=True, exist_ok=True)

    # Save
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/raw/ticketmaster_us_music_{timestamp}.json"
    

    with open(filename, "w") as f:
        json.dump(final_events, f, indent=2)

    # Summary
    table = Table(title="Ticketmaster Scrape Summary")
    table.add_column("Metric", style="cyan")
    table.add_column("Value", style="magenta")

    table.add_row("Total Raw Events", str(total_raw))
    table.add_row("Unique Events", str(len(final_events)))
    table.add_row("Total Requests Used", str(request_count))
    table.add_row("Daily Limit", str(DAILY_LIMIT))
    table.add_row("Saved File", filename)

    console.print("\n")
    console.print(table)
    console.print("\n[bold green]🎉 Scrape Completed Successfully! (Static Event Features Collected)[/bold green]\n")

    return final_events


if __name__ == "__main__":
    fetch_all_us_music_events()

🎵 Fetching ALL USA Music Events (Geohash Grid)

/workspaces/ticket-heroes/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Error 400 on geohash dr5 page 5: 
{"errors":[{"_links":{"about":{"href":"/discovery/v2/errors.html#DIS1035"}},"code":"DIS1035","detail":"API Limits 
Exceeded: Max paging depth exceeded. (page * size) must be less than 1,000","status":"400 BAD_REQUEST"}]}

Geohash dr5: 1000 unique events

Error 400 on geohash dr7 page 5: 
{"errors":[{"_links":{"about":{"href":"/discovery/v2/errors.html#DIS1035"}},"code":"DIS1035","detail":"API Limits 
Exceeded: Max paging depth exceeded. (page * size) must be less than 1,000","status":"400 BAD_REQUEST"}]}

...

Geohash 9yf: 270 unique events

Geohash 9yh: 644 unique events

Geohash 9yj: 174 unique events

                         Ticketmaster Scrape Summary                         
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric              ┃ Value                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Total Raw Events    │ 28675                                               │
│ Unique Events       │ 16050                                               │
│ Total Requests Used │ 371                                                 │
│ Daily Limit         │ 5000                                                │
│ Saved File          │ data/raw/ticketmaster_us_music_20251207_031136.json │
└─────────────────────┴─────────────────────────────────────────────────────┘

🎉 Scrape Completed Successfully! (Static Event Features Collected)

In [4]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from pathlib import Path
from rich.console import Console
from rich.table import Table
from rich import box

console = Console()

# Config
RAW_DIR = Path("data/raw")
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# Load + Normalize JSON
def load_event_json(path: str | Path) -> pd.DataFrame:
    """
    Loads raw Ticketmaster JSON and converts to flat DataFrame.
    """
    console.print(f"[cyan]Loading JSON:[/cyan] {path}")

    with open(path, "r") as f:
        data = json.load(f)

    df = pd.json_normalize(data)
    console.print(f"[green]Loaded {len(df)} events from {path.name}[/green]")

    return df


# Initial Basic Feature Engineering
def engineer_event_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds derived features:
    - datetime parsing
    - temporal components (days_until_event, days_since_onsale)
    - price range helpers
    - event type flags
    """

    console.print("[cyan]Engineering temporal and categorical features...[/cyan]")

    # --- Datetime Parsing ---
    df["event_date"] = pd.to_datetime(df.get("event_date"), errors="coerce", utc=True)
    df["onsale_date"] = pd.to_datetime(df.get("onsale_date"), errors="coerce", utc=True)

    # --- Temporal Features ---
    now = datetime.now(timezone.utc)
    
    # Days until event
    df["days_until_event"] = (df["event_date"] - now).dt.total_seconds() / (60*60*24)
    
    # Days since ticket went on sale (HIGH-VALUE FEATURE)
    df["days_since_onsale"] = (now - df["onsale_date"]).dt.total_seconds() / (60*60*24)
    
    # Basic date components
    df["event_year"] = df["event_date"].dt.year
    df["event_month"] = df["event_date"].dt.month
    df["event_day"] = df["event_date"].dt.day
    df["event_dayofweek"] = df["event_date"].dt.day_name() #

    # --- Categorical/Flag Features ---
    df["is_festival"] = df["event_name"].str.contains("festival", case=False, na=False).astype(int)
    df["is_tour"] = df["event_name"].str.contains("tour", case=False, na=False).astype(int)

    # --- Price Features (Anchors) ---
    # Convert capacity to numeric here as well for early feature completion
    df["capacity"] = pd.to_numeric(df.get("capacity"), errors="coerce")
    
    # Midpoint of the primary ticketmaster price range
    df["price_mid"] = df[["price_min", "price_max"]].mean(axis=1)

    console.print("[green]Finished feature engineering.[/green]")
    return df


# Cleaning + Standardization
def clean_event_panel(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standard clean-up:
    - drop duplicates
    - enforce numerical types
    """

    console.print("[cyan]Cleaning dataset...[/cyan]")

    # Drop events without ID (required for joins later)
    df = df.dropna(subset=["event_id"])
    
    # --- Deduplication ---
    before = len(df)
    # Deduplicate by event_id, keeping the last record found (assuming the later scrape is slightly fresher)
    df = df.drop_duplicates(subset=["event_id"], keep='last')
    after = len(df)

    console.print(f"[yellow]Removed {before - after} duplicates.[/yellow]")

    # --- Numeric Type Enforcement ---
    num_cols = [
        "latitude", "longitude", "capacity",
        "price_min", "price_max", "price_mid",
        "days_until_event", "days_since_onsale"
    ]

    for col in num_cols:
        if col in df.columns:
            # Use errors="coerce" to turn non-numeric values into NaN
            df[col] = pd.to_numeric(df[col], errors="coerce")

    console.print("[green]Dataset cleaned.[/green]")
    return df


# Quick Summary Table
def summarize_panel(df: pd.DataFrame):
    console.print("\n[bold cyan]Event Panel Summary[/bold cyan]")

    # --- Summary Table ---
    table = Table(title="Event Panel Statistics", box=box.SIMPLE_HEAVY)
    table.add_column("Metric", style="yellow")
    table.add_column("Value", style="green")

    table.add_row("Total Unique Events", str(len(df)))
    table.add_row("Unique Artists", str(df["artist_name"].nunique()))
    table.add_row("Unique Cities", str(df["city"].nunique()))
    table.add_row("Average Days Until Event", f"{df['days_until_event'].mean():.2f} days")
    table.add_row("Median Days Since Onsale", f"{df['days_since_onsale'].median():.2f} days")
    table.add_row("Median Ticket Price (mid)", f"${df['price_mid'].median():.2f}")
    table.add_row("Event Date Span", f"{df['event_date'].min().strftime('%Y-%m-%d')} → {df['event_date'].max().strftime('%Y-%m-%d')}")

    console.print(table)


# Main Pipeline (UPDATED to handle multiple files)
def build_event_panel(raw_json_paths: list[str | Path]):
    """
    Full pipeline:
      Multiple JSONs → Concatenated DataFrame → Cleaned Event Panel → CSV & Parquet
    """
    
    all_dfs = []
    
    # 1. Load and Concatenate All Files
    for path in raw_json_paths:
        path = Path(path)
        if not path.exists():
            console.print(f"[bold red]ERROR:[/bold red] File not found at {path}")
            continue
        all_dfs.append(load_event_json(path))

    if not all_dfs:
        console.print("[bold red]Pipeline halted: No valid files loaded.[/bold red]")
        return pd.DataFrame()

    df = pd.concat(all_dfs, ignore_index=True)
    console.print(f"\n[bold green]Concatenated {len(all_dfs)} files into {len(df)} total rows.[/bold green]")

    # 2. Engineer, Clean, and Summarize
    df = engineer_event_features(df)
    df = clean_event_panel(df)

    # 3. Save Output
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = OUT_DIR / f"event_panel_{timestamp}.csv"
    pq_path = OUT_DIR / f"event_panel_{timestamp}.parquet"

    df.to_csv(csv_path, index=False)
    df.to_parquet(pq_path, index=False)

    summarize_panel(df)

    console.print(f"\n[bold green]Saved CSV →[/bold green] {csv_path}")
    console.print(f"[bold green]Saved Parquet →[/bold green] {pq_path}\n")

    return df


# --- EXAMPLE USAGE (For Notebook 1) ---
if __name__ == "__main__":
    # Assuming you have two files named like the output of your scraper:
    file1 = RAW_DIR / "ticketmaster_events_20251207_023443.json"
    file2 = RAW_DIR / "ticketmaster_us_music_20251207_031136.json"
    
    # Create dummy files for demonstration if they don't exist
    if not file1.exists():
         file1.parent.mkdir(parents=True, exist_ok=True)
         with open(file1, 'w') as f:
             json.dump([{"event_id": "a", "event_date": "2026-06-01T10:00:00Z", "onsale_date": "2025-01-01T10:00:00Z", "price_min": 10, "price_max": 100, "city": "NYC", "state_code": "NY"}], f)
    if not file2.exists():
         with open(file2, 'w') as f:
             json.dump([{"event_id": "b", "event_date": "2026-07-01T10:00:00Z", "onsale_date": "2025-02-01T10:00:00Z", "price_min": 20, "price_max": 200, "city": "LA", "state_code": "CA"}], f)

    # Pass the list of files to the main pipeline function
    df_panel = build_event_panel([file1, file2])

Loading JSON: data/raw/ticketmaster_events_20251207_023443.json

Loaded 1000 events from ticketmaster_events_20251207_023443.json

Loading JSON: data/raw/ticketmaster_us_music_20251207_031136.json

Loaded 16050 events from ticketmaster_us_music_20251207_031136.json

Concatenated 2 files into 17050 total rows.

...

Finished feature engineering.

Cleaning dataset...

Removed 0 duplicates.

Dataset cleaned.

ArrowKeyError: No type extension with name arrow.py_extension_type found

In [ ]:
import requests
import time
import json
from datetime import datetime
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn
from rich.table import Table
from pathlib import Path

# Ensure directory exists
Path("data/raw").mkdir(parents=True, exist_ok=True)

# ---------------------------------------
# Rich Console
# ---------------------------------------
console = Console()

# ---------------------------------------
# Ticketmaster API Config
# ---------------------------------------
API_KEY = "MhlwpLEgaTMO0kxeTAZnx1tLsUeOAGEC"
BASE = "https://app.ticketmaster.com/discovery/v2/events.json"

# Segments & Classifications
SEGMENT_MUSIC = "KZFzniwnSyZfZ7v7nJ"
SEGMENT_SPORTS = "KZFzniwnSyZfZ7v7nE"       # TM sports segment ID
CLASSIFICATION_FESTIVAL = "KZazBEonSMnZfZ7vF1" # Festival classification (not a segment)

DAILY_LIMIT = 5000
SAFE_RPS = 1.67
SLEEP_TIME = 1 / SAFE_RPS  # ~0.6 seconds

# ---------------------------------------
# USA Geohash Coverage Grid (36 cells)
# ---------------------------------------
US_GEOHASHES = [
    "dr5","dr7","dr9","drc","drf","drg",
    "dj8","djd","djf","djg","djk",
    "dp3","dp5","dp8","dp9","dpe","dph",
    "dq2","dq5","dq8","dq9",
    "9q5","9q8","9qb","9qc","9qh","9qk",
    "9xj","9xk","9xm","9xs",
    "9yf","9yh","9yj"
]

# ---------------------------------------
# Global Request Counter
# ---------------------------------------
request_count = 200

# =======================================
# METADATA EXTRACTION (unchanged)
# =======================================
def extract_metadata(raw_event):
    event_id = raw_event.get("id")
    event_name = raw_event.get("name")
    event_url = raw_event.get("url")

    event_date = raw_event.get("dates", {}).get("start", {}).get("dateTime")
    timezone = raw_event.get("dates", {}).get("timezone")
    event_status = raw_event.get("dates", {}).get("status", {}).get("code")
    onsale_date = raw_event.get("sales", {}).get("public", {}).get("startDateTime")

    embedded = raw_event.get("_embedded", {})

    attractions = embedded.get("attractions", [{}])
    artist = attractions[0] if attractions else {}

    artist_id = artist.get("id")
    artist_name = artist.get("name")
    artist_url = artist.get("url")

    venue_list = embedded.get("venues", [{}])
    venue = venue_list[0] if venue_list else {}

    venue_id = venue.get("id")
    venue_name = venue.get("name")

    city = venue.get("city", {}).get("name")
    state_code = venue.get("state", {}).get("stateCode")
    postal_code = venue.get("postalCode")
    country_code = venue.get("country", {}).get("countryCode")

    location = venue.get("location", {})
    latitude = location.get("latitude")
    longitude = location.get("longitude")

    capacity = venue.get("capacity")

    price_ranges = raw_event.get("priceRanges", [{}])
    price_min = price_ranges[0].get("min")
    price_max = price_ranges[0].get("max")
    currency = price_ranges[0].get("currency")

    class_info = raw_event.get("classifications", [{}])[0]

    segment = class_info.get("segment", {}).get("name")
    genre = class_info.get("genre", {}).get("name")
    subgenre = class_info.get("subGenre", {}).get("name")
    type_name = class_info.get("type", {}).get("name")
    subtype = class_info.get("subType", {}).get("name")

    promoter = raw_event.get("promoter", {})
    promoter_name = promoter.get("name")
    promoter_id = promoter.get("id")

    seatmap_url = raw_event.get("seatmap", {}).get("staticUrl")

    return {
        "event_id": event_id,
        "event_name": event_name,
        "event_url": event_url,
        "event_date": event_date,
        "timezone": timezone,
        "event_status": event_status,
        "onsale_date": onsale_date,
        "artist_id": artist_id,
        "artist_name": artist_name,
        "artist_url": artist_url,
        "venue_id": venue_id,
        "venue_name": venue_name,
        "city": city,
        "state_code": state_code,
        "postal_code": postal_code,
        "country_code": country_code,
        "latitude": latitude,
        "longitude": longitude,
        "capacity": capacity,
        "price_min": price_min,
        "price_max": price_max,
        "currency": currency,
        "segment": segment,
        "genre": genre,
        "subgenre": subgenre,
        "type": type_name,
        "subtype": subtype,
        "promoter_name": promoter_name,
        "promoter_id": promoter_id,
        "seatmap_url": seatmap_url
    }

# =======================================
# Request Wrapper
# =======================================
def make_request(url, params):
    global request_count
    request_count += 1

    if request_count % 50 == 0:
        console.print(f"[yellow]Request count: {request_count}/{DAILY_LIMIT}[/yellow]")

    if request_count >= DAILY_LIMIT * 0.9:
        console.print(f"[bold red]⚠ NEAR DAILY LIMIT: {request_count} requests[/bold red]")

    return requests.get(url, params=params)

# =======================================
# Fetch Events per Geohash per Category
# =======================================
def fetch_events_for_geohash(geohash, segment_id=None, classification_id=None):
    cleaned_events = []
    total_raw = 0
    page = 0

    while True:
        params = {
            "apikey": API_KEY,
            "geoPoint": geohash,
            "radius": 200,
            "size": 200,
            "page": page
        }

        # Segment-based queries (music, sports)
        if segment_id:
            params["segmentId"] = segment_id

        # Classification-based queries (festivals)
        if classification_id:
            params["classificationId"] = classification_id

        response = make_request(BASE, params)

        if response.status_code == 429:
            console.print("[bold red]429 rate limit → sleeping 5s[/bold red]")
            time.sleep(5)
            continue

        if response.status_code != 200:
            console.print(f"[red]Error {response.status_code} at {geohash} page {page}[/red]")
            break

        data = response.json()

        if "_embedded" not in data or "events" not in data["_embedded"]:
            break

        raw_events = data["_embedded"]["events"]
        total_raw += len(raw_events)

        for raw_event in raw_events:
            cleaned_events.append(extract_metadata(raw_event))

        total_pages = data["page"]["totalPages"]
        if page >= total_pages - 1:
            break

        page += 1
        time.sleep(SLEEP_TIME)

    return cleaned_events, total_raw

# =======================================
# Multi-segment Harvester
# =======================================
SEGMENTS = {
    "music": {"segment": SEGMENT_MUSIC, "classification": None},
    "sports": {"segment": SEGMENT_SPORTS, "classification": None},
    "festivals": {"segment": None, "classification": CLASSIFICATION_FESTIVAL},
}

def fetch_multi_segment():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for seg_name, cfg in SEGMENTS.items():
        console.print(f"\n[bold cyan]🔎 Fetching {seg_name.upper()} events...[/bold cyan]\n")

        all_events = {}
        total_raw = 0

        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            BarColumn(),
            TextColumn("{task.completed}/{task.total} geohashes done"),
            transient=True,
        ) as progress:

            task = progress.add_task(f"{seg_name} geohashes...", total=len(US_GEOHASHES))

            for gh in US_GEOHASHES:
                events, raw_count = fetch_events_for_geohash(
                    gh,
                    segment_id=cfg["segment"],
                    classification_id=cfg["classification"]
                )
                total_raw += raw_count

                for ev in events:
                    all_events[ev["event_id"]] = ev

                progress.update(task, advance=1)

        final_events = list(all_events.values())

        out_path = f"data/raw/ticketmaster_{seg_name}_{timestamp}.json"
        with open(out_path, "w") as f:
            json.dump(final_events, f, indent=2)

        table = Table(title=f"{seg_name.upper()} Summary")
        table.add_column("Metric")
        table.add_column("Value")

        table.add_row("Total Raw Events", str(total_raw))
        table.add_row("Unique Events", str(len(final_events)))
        table.add_row("Requests Used", str(request_count))
        table.add_row("Saved File", out_path)

        console.print(table)
        console.print(f"[green]Finished {seg_name}![/green]")

    console.print("\n[bold green]🎉 Multi-segment scrape complete![/bold green]\n")

if __name__ == "__main__":
    fetch_multi_segment()


🔎 Fetching MUSIC events...

Output()

Error 400 at dr5 page 5

Error 400 at dr7 page 5

Error 400 at dr9 page 5

Error 400 at drc page 5

In [ ]:
import requests
import time
import json
from datetime import datetime
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn
from rich.table import Table
from pathlib import Path

# ---------------------------------------
# Ensure directory exists
# ---------------------------------------
Path("data/raw").mkdir(parents=True, exist_ok=True)

console = Console()

# ---------------------------------------
# Ticketmaster API Config
# ---------------------------------------
API_KEY = "MhlwpLEgaTMO0kxeTAZnx1tLsUeOAGEC"
BASE = "https://app.ticketmaster.com/discovery/v2/events.json"

SEGMENT_MUSIC = "KZFzniwnSyZfZ7v7nJ"
SEGMENT_SPORTS = "KZFzniwnSyZfZ7v7nE"
SEGMENT_FESTIVALS = "KZFzniwnSyZfZ7v7n1"  # Official "Fairs & Festivals"

DAILY_LIMIT = 5000
SAFE_RPS = 1.67
SLEEP_TIME = 1 / SAFE_RPS  # ~0.6 seconds

SEGMENTS = {
    "music": SEGMENT_MUSIC,
    "sports": SEGMENT_SPORTS,
    "festivals": SEGMENT_FESTIVALS,
}

# ---------------------------------------
# USA Geohash Coverage Grid
# ---------------------------------------
US_GEOHASHES = [
    "dr5","dr7","dr9","drc","drf","drg",
    "dj8","djd","djf","djg","djk",
    "dp3","dp5","dp8","dp9","dpe","dph",
    "dq2","dq5","dq8","dq9",
    "9q5","9q8","9qb","9qc","9qh","9qk",
    "9xj","9xk","9xm","9xs",
    "9yf","9yh","9yj"
]

# ---------------------------------------
# Global Request Counter
# ---------------------------------------
request_count = 0


# ==========================================================
# METADATA EXTRACTION (safe, fault-tolerant)
# ==========================================================
def extract_metadata(raw_event):
    embedded = raw_event.get("_embedded", {})

    # Artist
    artist = embedded.get("attractions", [{}])[0] or {}
    venue = embedded.get("venues", [{}])[0] or {}

    # Price
    price = (raw_event.get("priceRanges") or [{}])[0]

    # Classification
    class_info = (raw_event.get("classifications") or [{}])[0]

    return {
        "event_id": raw_event.get("id"),
        "event_name": raw_event.get("name"),
        "event_url": raw_event.get("url"),
        "event_date": raw_event.get("dates", {}).get("start", {}).get("dateTime"),
        "timezone": raw_event.get("dates", {}).get("timezone"),
        "event_status": raw_event.get("dates", {}).get("status", {}).get("code"),
        "onsale_date": raw_event.get("sales", {}).get("public", {}).get("startDateTime"),

        # Artist
        "artist_id": artist.get("id"),
        "artist_name": artist.get("name"),
        "artist_url": artist.get("url"),

        # Venue
        "venue_id": venue.get("id"),
        "venue_name": venue.get("name"),
        "city": venue.get("city", {}).get("name"),
        "state_code": venue.get("state", {}).get("stateCode"),
        "postal_code": venue.get("postalCode"),
        "country_code": venue.get("country", {}).get("countryCode"),

        "latitude": venue.get("location", {}).get("latitude"),
        "longitude": venue.get("location", {}).get("longitude"),
        "capacity": venue.get("capacity"),

        # Price
        "price_min": price.get("min"),
        "price_max": price.get("max"),
        "currency": price.get("currency"),

        # Classification
        "segment": class_info.get("segment", {}).get("name"),
        "genre": class_info.get("genre", {}).get("name"),
        "subgenre": class_info.get("subGenre", {}).get("name"),
        "type": class_info.get("type", {}).get("name"),
        "subtype": class_info.get("subType", {}).get("name"),

        # Promoter
        "promoter_name": raw_event.get("promoter", {}).get("name"),
        "promoter_id": raw_event.get("promoter", {}).get("id"),

        # Seatmap
        "seatmap_url": raw_event.get("seatmap", {}).get("staticUrl"),
    }


# ==========================================================
# Request Wrapper with Count + Warning
# ==========================================================
def make_request(url, params):
    global request_count
    request_count += 1

    if request_count % 50 == 0:
        console.print(f"[yellow]Request count: {request_count}/{DAILY_LIMIT}[/yellow]")

    if request_count >= DAILY_LIMIT * 0.9:
        console.print(f"[bold red]⚠ WARNING: near daily limit ({request_count})[/bold red]")

    return requests.get(url, params=params)


# ==========================================================
# Robust Geohash Fetcher (with radius fallback for 400 errors)
# ==========================================================
def fetch_events_for_geohash(geohash, segment_id):
    cleaned_events = []
    total_raw = 0
    page = 0

    RADII = [200, 100, 50, 25]

    while True:
        success = False

        for radius in RADII:
            params = {
                "apikey": API_KEY,
                "segmentId": segment_id,
                "geoPoint": geohash,
                "radius": radius,
                "size": 200,
                "page": page,
            }

            response = make_request(BASE, params)

            # Rate limit
            if response.status_code == 429:
                console.print("[red]429: Sleeping 5s[/red]")
                time.sleep(5)
                continue

            # Success
            if response.status_code == 200:
                data = response.json()
                events = data.get("_embedded", {}).get("events", [])
                total_raw += len(events)

                for raw_event in events:
                    cleaned_events.append(extract_metadata(raw_event))

                total_pages = data["page"]["totalPages"]

                if page >= total_pages - 1:
                    return cleaned_events, total_raw

                page += 1
                time.sleep(SLEEP_TIME)
                success = True
                break

            # 400 fallback
            if response.status_code == 400:
                console.print(
                    f"[yellow]400 at {geohash}, page {page}, radius {radius} → retry smaller radius[/yellow]"
                )
                continue

            # Hard fail (404, 500, etc)
            console.print(f"[red]Error {response.status_code}: {response.text}[/red]")
            return cleaned_events, total_raw

        # All radii failed → skip geohash page
        if not success:
            console.print(
                f"[red]❌ All radii failed at {geohash} page {page}. Skipping.[/red]"
            )
            return cleaned_events, total_raw


# ==========================================================
# Full Segment Harvester
# ==========================================================
def harvest_segment(segment_name, segment_id):
    console.print(f"\n[bold cyan]📡 Harvesting segment: {segment_name}[/bold cyan]\n")

    all_events = {}
    total_raw = 0

    with Progress(
        SpinnerColumn(),
        TextColumn("{task.description}"),
        BarColumn(),
        TextColumn("{task.completed}/{task.total} geohashes"),
    ) as progress:

        task = progress.add_task(f"Scanning {segment_name}...", total=len(US_GEOHASHES))

        for gh in US_GEOHASHES:
            events, raw = fetch_events_for_geohash(gh, segment_id)
            total_raw += raw

            for ev in events:
                all_events[ev["event_id"]] = ev  # dedupe

            progress.update(task, advance=1)

    console.print(
        f"[green]Finished {segment_name}: {len(all_events)} unique events ({total_raw} raw)[/green]"
    )
    return all_events


# ==========================================================
# MAIN: Run all segments
# ==========================================================
def run_full_harvest():
    console.print("\n[bold magenta]🚀 Starting FULL Ticketmaster US Event Harvest[/bold magenta]\n")

    all_events = {}

    for name, seg_id in SEGMENTS.items():
        seg_events = harvest_segment(name, seg_id)
        all_events.update(seg_events)

    final_events = list(all_events.values())

    # Save
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/raw/ticketmaster_us_events_{timestamp}.json"

    with open(filename, "w") as f:
        json.dump(final_events, f, indent=2)

    # Summary
    table = Table(title="Ticketmaster Harvest Summary")
    table.add_column("Metric", style="cyan")
    table.add_column("Value", style="magenta")

    table.add_row("Total Unique Events", str(len(final_events)))
    table.add_row("Total Requests Used", str(request_count))
    table.add_row("Daily Limit", str(DAILY_LIMIT))
    table.add_row("Saved File", filename)

    console.print("\n")
    console.print(table)
    console.print("\n[bold green]🎉 Harvest Complete![/bold green]\n")

    return final_events


if __name__ == "__main__":
    run_full_harvest()


🚀 Starting FULL Ticketmaster US Event Harvest

📡 Harvesting segment: music

Output()

400 at dr5, page 5, radius 200 → retry smaller radius

400 at dr5, page 5, radius 100 → retry smaller radius

...

400 at dr7, page 5, radius 50 → retry smaller radius

400 at dr7, page 5, radius 25 → retry smaller radius

❌ All radii failed at dr7 page 5. Skipping.

400 at dr9, page 5, radius 200 → retry smaller radius

400 at dr9, page 5, radius 100 → retry smaller radius